# **Laboratorio 3**

Daniel Arturo López Sánchez, 20190285

## **Parte 1**

In [1]:
import pandas as pd

In [2]:
import mysql.connector
from mysql.connector import Error

def create_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [3]:
host = "relational.fit.cvut.cz"
user = "guest"
pwd = "relational"
connection = create_connection(host_name = host, user_name =user ,user_password = pwd)

Connection to MySQL DB successful


 1. Obtener el nombre, apellido, salario, título y vigencia del mismo para el empleado 12557

In [4]:
query = """ SELECT
                A.first_name,
                A.last_name,
                A.emp_no,
                B.title,
                C.from_date,
                C.to_date,
                C.salary
         FROM employee.employees A  
         LEFT JOIN employee.titles B on A.emp_no = B.emp_no
         LEFT JOIN employee.salaries C on A.emp_no = C.emp_no
         WHERE A.emp_no = 12557
"""
emp_12557 = pd.read_sql(query,connection)
emp_12557

,first_name,last_name,emp_no,title,from_date,to_date,salary
0,Stafford,Kopetz,12557,Senior Engineer,1988-03-20,1989-03-20,41840
1,Stafford,Kopetz,12557,Senior Engineer,1989-03-20,1990-03-20,46021
2,Stafford,Kopetz,12557,Senior Engineer,1990-03-20,1991-03-20,48699
3,Stafford,Kopetz,12557,Senior Engineer,1991-03-20,1992-03-19,51610
4,Stafford,Kopetz,12557,Senior Engineer,1992-03-19,1993-03-19,51351
5,Stafford,Kopetz,12557,Senior Engineer,1993-03-19,1994-03-19,51660
6,Stafford,Kopetz,12557,Senior Engineer,1994-03-19,1995-03-19,52878
7,Stafford,Kopetz,12557,Senior Engineer,1995-03-19,1996-03-18,54053
8,Stafford,Kopetz,12557,Senior Engineer,1996-03-18,1997-03-18,57950
9,Stafford,Kopetz,12557,Senior Engineer,1997-03-18,1998-03-18,58055


2. Encontrar el último departamento en el que trabajó el empleado con la siguiente información del mismo: nombre, apellido, número de empleado, fecha de nacimiento, código de departamento, nombre del departamento, fecha cuando inició su trabajo en ese departamento y la fecha cuando concluyó su trabajo

In [5]:
subquery = """
    SELECT
        emp_no,
        dept_no,
        to_date,
        MAX(from_date) fecha_max
    FROM employee.dept_emp
    GROUP BY 1,2,3 LIMIT 100
"""

pd.read_sql(subquery,connection)

,emp_no,dept_no,to_date,fecha_max
0,10001,d005,9999-01-01,1986-06-26
1,10002,d007,9999-01-01,1996-08-03
2,10003,d004,9999-01-01,1995-12-03
3,10004,d004,9999-01-01,1986-12-01
4,10005,d003,9999-01-01,1989-09-12
...,...,...,...,...
95,10088,d007,1992-03-21,1988-09-02
96,10088,d009,9999-01-01,1992-03-21
97,10089,d007,9999-01-01,1989-01-10
98,10090,d005,1999-05-07,1986-03-14


In [6]:
%%time
query = """
        SELECT 
        B.first_name,
        B.last_name,
        B.emp_no,
        B.birth_date,
        A.dept_no,
        C.dept_name,
        B.hire_date,
        A.to_date 
        FROM (        SELECT
            emp_no,
            dept_no,
            to_date,
            MAX(from_date) fecha_max
        FROM employee.dept_emp
        GROUP BY 1,2,3
        LIMIT 100) A
        LEFT JOIN employee.employees B on A.emp_no = B.emp_no 
         LEFT JOIN employee.departments C on A.dept_no = C.dept_no 
        LIMIT 10
    

"""
pd.read_sql(query,connection)

Wall time: 1.14 s


,first_name,last_name,emp_no,birth_date,dept_no,dept_name,hire_date,to_date
0,Georgi,Facello,10001,1953-09-02,d005,Development,1986-06-26,9999-01-01
1,Bezalel,Simmel,10002,1964-06-02,d007,Sales,1985-11-21,9999-01-01
2,Parto,Bamford,10003,1959-12-03,d004,Production,1986-08-28,9999-01-01
3,Chirstian,Koblick,10004,1954-05-01,d004,Production,1986-12-01,9999-01-01
4,Kyoichi,Maliniak,10005,1955-01-21,d003,Human Resources,1989-09-12,9999-01-01
5,Anneke,Preusig,10006,1953-04-20,d005,Development,1989-06-02,9999-01-01
6,Tzvetan,Zielinski,10007,1957-05-23,d008,Research,1989-02-10,9999-01-01
7,Saniya,Kalloufi,10008,1958-02-19,d005,Development,1994-09-15,2000-07-31
8,Sumant,Peac,10009,1952-04-19,d006,Quality Management,1985-02-18,9999-01-01
9,Duangkaew,Piveteau,10010,1963-06-01,d004,Production,1989-08-24,2000-06-26


3. Encontrar el salario máximo de cada empleado utilizando la función Partition By con toda la información del empleado y de su salario

In [7]:
%%time
query = """
    SELECT *
    FROM (
        SELECT
            A.emp_no
            ,A.birth_date
            ,A.first_name
            ,A.last_name
            ,A.gender
            ,B.salary AS max_salary
            ,ROW_NUMBER() OVER (PARTITION BY A.emp_no ORDER BY B.salary DESC) AS seq
        FROM employee.employees A
        LEFT JOIN employee.salaries B ON B.emp_no = A.emp_no
        GROUP BY 1,2,3,4,5,6
    ) A
    WHERE seq = 1
    LIMIT 100
"""
pd.read_sql(query,connection)

Wall time: 10.4 s


,emp_no,birth_date,first_name,last_name,gender,max_salary,seq
0,10001,1953-09-02,Georgi,Facello,M,88958,1
1,10002,1964-06-02,Bezalel,Simmel,F,72527,1
2,10003,1959-12-03,Parto,Bamford,M,43699,1
3,10004,1954-05-01,Chirstian,Koblick,M,74057,1
4,10005,1955-01-21,Kyoichi,Maliniak,M,94692,1
...,...,...,...,...,...,...,...
95,10096,1954-09-16,Jayson,Mandell,M,68612,1
96,10097,1952-02-27,Remzi,Waschkowski,M,70161,1
97,10098,1961-09-23,Sreekrishna,Servieres,F,56202,1
98,10099,1956-05-25,Valter,Sullins,F,98538,1


## **Parte 2**

In [27]:
import sqlite3
con = sqlite3.connect('sql-murder-mystery.db')

### Overview de las tablas

In [28]:
query = """
SELECT name 
  FROM sqlite_master
 where type = 'table'
 
 """
pd.read_sql(query, con)

,name
0,crime_scene_report
1,drivers_license
2,person
3,facebook_event_checkin
4,interview
5,get_fit_now_member
6,get_fit_now_check_in
7,income
8,solution


In [29]:
query = "select * from crime_scene_report;"
crime_scene_report = pd.read_sql(query,con)
crime_scene_report.head(15)

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City
5,20180115,theft,Big Bully stole my lunch money!,Chicago
6,20180115,fraud,"Lorem ipsum dolor sit amet, consectetur adipis...",Seattle
7,20170712,theft,"A lone hunter stalks the night, firing arrows ...",SQL City
8,20170820,arson,"Wield the Hammer of Sol with honor, Titan, it ...",SQL City
9,20171110,robbery,The Gjallarhorn shoulder-mounted rocket system...,SQL City


In [30]:
query = "select * from drivers_license;"
drivers_license = pd.read_sql(query,con)
drivers_license.head(15)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,100280,72,57,brown,red,male,P24L4U,Acura,MDX
1,100460,63,72,brown,brown,female,XF02T6,Cadillac,SRX
2,101029,62,74,green,green,female,VKY5KR,Scion,xB
3,101198,43,54,amber,brown,female,Y5NZ08,Nissan,Rogue
4,101255,18,79,blue,grey,female,5162Z1,Lexus,GS
5,101494,48,55,blue,red,female,81X1N7,Kia,Sportage
6,101568,53,78,brown,blue,male,SJ57LL,Saab,9-5
7,101586,57,70,amber,brown,male,31U1KE,Jaguar,XK
8,101611,40,65,blue,white,female,5O8VW7,GMC,Sierra Denali
9,101613,81,58,brown,grey,female,5PCQB5,GMC,Sierra 2500


In [31]:
query = "select * from person;"
person = pd.read_sql(query,con)
person.head(15)

,id,name,license_id,address_number,address_street_name,ssn
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684
5,10027,Antione Godbolt,439509,2431,Zelham Dr,491650087
6,10034,Kyra Buen,920494,1873,Sleigh Dr,332497972
7,10039,Francesco Agundez,278151,736,Buswell Dr,861079251
8,10095,Leslie Thate,729987,2772,Camellia Park Circle,127944356
9,10122,Alva Conkel,779002,116,Diversey Circle,148521773


In [32]:
query = "select * from facebook_event_checkin;"
fb_event_checkin = pd.read_sql(query,con)
fb_event_checkin.head(15)

,person_id,event_id,event_name,date
0,28508,5880,Nudists are people who wear one-button suits.\n,20170913
1,63713,3865,but that's because it's the best book on anyth...,20171009
2,63713,3999,"If Murphy's Law can go wrong, it will.\n",20170502
3,63713,6436,Old programmers never die. They just branch t...,20170926
4,82998,4470,Help a swallow land at Capistrano.\n,20171022
5,82998,6186,Good day to let down old friends who need help.\n,20170921
6,82998,7364,"Among economists, the real world is of...",20170320
7,31811,8542,-- Herbert Prochnow\n,20171203
8,31811,4387,"The Pig, if I am not mistaken,\n",20180106
9,61201,9621,responsibility at the other.\n,20180418


In [33]:
query = "select * from interview;"
interview = pd.read_sql(query,con)
interview.head(15)

,person_id,transcript
0,28508,‘I deny it!’ said the March Hare.\n
1,63713,\n
2,86208,"way, and the whole party swam to the shore.\n"
3,35267,"lessons in here? Why, there’s hardly room for ..."
4,33856,\n
5,82799,\n
6,66493,would be offended again.\n
7,46263,"‘So he did, so he did,’ said the Gryphon, sigh..."
8,54263,very glad that it was over at last: ‘and I do ...
9,62764,come to the end: then stop.’\n


In [34]:
query = "select * from get_fit_now_member;"
get_fit_now_member = pd.read_sql(query,con)
get_fit_now_member.head(15)

,id,person_id,name,membership_start_date,membership_status
0,NL318,65076,Everette Koepke,20170926,gold
1,AOE21,39426,Noe Locascio,20171005,regular
2,2PN28,63823,Jeromy Heitschmidt,20180215,silver
3,0YJ24,80651,Waneta Wellard,20171206,gold
4,3A08L,32858,Mei Bianchin,20170401,silver
5,6W6I2,69510,Paris Accurso,20170613,regular
6,5Y28Y,15218,Millicent Yessios,20180205,gold
7,156A7,79701,Marcelino Leard,20180428,silver
8,A5N3S,24541,Mary Cameron,20170501,gold
9,X0643,15247,Shondra Ledlow,20170521,silver


In [35]:
query = "select * from get_fit_now_check_in;"
get_fit_now_check_in = pd.read_sql(query,con)
get_fit_now_check_in.head(15)

,membership_id,check_in_date,check_in_time,check_out_time
0,NL318,20180212,329,365
1,NL318,20170811,469,920
2,NL318,20180429,506,554
3,NL318,20180128,124,759
4,NL318,20171027,418,1019
5,NL318,20171018,608,1074
6,NL318,20171127,850,954
7,NL318,20170627,591,715
8,NL318,20180114,777,813
9,NL318,20171107,148,460


In [36]:
query = "select * from income;"
income = pd.read_sql(query,con)
income.head(15)

,ssn,annual_income
0,100009868,52200
1,100169584,64500
2,100300433,74400
3,100355733,35900
4,100366269,73000
5,100593316,16100
6,100626193,10800
7,100739881,53800
8,100749089,61200
9,100958115,51200


Informe de la escena del crimen: asesinato el 15 de enero de 2018 en SQL City

In [50]:
query = """ SELECT description
         FROM crime_scene_report AS A  
         WHERE A.date = 20180115 AND A.type = 'murder' AND A.city = 'SQL City'
"""
pd.read_sql(query,con)

,description
0,Security footage shows that there were 2 witne...


Las camaras de seguridad muestran que hubieron 2 testigos. Uno vive en la ultima casa de NorthwesterN Dr y la otra se llama Annabel y vive en la Franklin ave

In [58]:
query="""SELECT *,
        MAX(address_number) last_house
        FROM person
        WHERE address_street_name="Northwestern Dr"     
"""
pd.read_sql(query,con)

,id,name,license_id,address_number,address_street_name,ssn,last_house
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949,4919


Testigo 1: Morty Schapiro

In [59]:
query=""" SELECT * FROM person 
        WHERE address_street_name="Franklin Ave" AND name="Annabel Miller"
"""
pd.read_sql(query,con)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [60]:
query=''' SELECT *
        FROM interview
        WHERE person_id = 14887 OR person_id = 16371
'''
pd.read_sql(query,con)

,person_id,transcript
0,14887,I heard a gunshot and then saw a man run out. ...
1,16371,"I saw the murder happen, and I recognized the ..."


**Nuevas pistas:** El asesino va a *Get fit now gym*, fue visto el 9 de enero, su bolsa tiene el numero "48Z", su membresia es "gold" y su placa es la "H42W"

In [62]:
query=""" SELECT *
    FROM get_fit_now_check_in
    WHERE check_in_date = 20180109
"""
pd.read_sql(query,con)

,membership_id,check_in_date,check_in_time,check_out_time
0,X0643,20180109,957,1164
1,UK1F2,20180109,344,518
2,XTE42,20180109,486,1124
3,1AE2H,20180109,461,944
4,6LSTG,20180109,399,515
5,7MWHJ,20180109,273,885
6,GE5Q8,20180109,367,959
7,48Z7A,20180109,1600,1730
8,48Z55,20180109,1530,1700
9,90081,20180109,1600,1700


Buscando info de las dos personas con membership id de 48Z...

In [67]:
query=""" SELECT *
        FROM get_fit_now_member
        WHERE id="48Z55" OR id="48Z7A"
"""
pd.read_sql(query,con)

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold
1,48Z7A,28819,Joe Germuska,20160305,gold


Ambos son de membresia gold. Buscaremos el id de la licensia para luego ver las placas

In [74]:
query="""SELECT * 
        FROM person
        WHERE id="67318"  OR id="28819"
"""
pd.read_sql(query,con)

,id,name,license_id,address_number,address_street_name,ssn
0,28819,Joe Germuska,173289,111,Fisk Rd,138909730
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [75]:
query=""" SELECT * 
        FROM drivers_license
        WHERE id="173286"  OR id="423327"
"""
pd.read_sql(query,con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS


Sospechoso: Jeremy Bowers

In [77]:
cursor = con.cursor()
sql = "INSERT INTO solution VALUES (1, 'Jeremy Bowers')"
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]

"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villian behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."

**Buscando quien esta detras del asesinato**

In [78]:
query="""SELECT * 
FROM interview
WHERE person_id = "67318"
"""
pd.read_sql(query,con)

,person_id,transcript
0,67318,I was hired by a woman with a lot of money. I ...


**Pistas:** Es mujer, tiene cabello rojo, mide entre 5'5 y 5'7, su carro es un Tesla Model S y asistio a la sinfonia 3 veces en diciembre 2017

In [79]:
query="""
    SELECT *
    FROM (
        SELECT
            A.height,
            A.hair_color,
            A.gender,
            A.car_model,
            B.id,
            B.name,
            C.event_name
        FROM drivers_license A
        LEFT JOIN person B on A.id = B.license_id
        LEFT JOIN facebook_event_checkin C on B.id = C.person_id
        GROUP BY 1,2,3,4,5,6,7
    ) A
    WHERE (A.hair_color="red" AND A.gender="female" AND A.car_model="Model S") AND A.height BETWEEN 65 AND 67
"""
pd.read_sql(query,con)

,height,hair_color,gender,car_model,id,name,event_name
0,65,red,female,Model S,78881,Red Korb,None
1,66,red,female,Model S,90700,Regina George,None
2,66,red,female,Model S,99716,Miranda Priestly,SQL Symphony Concert


In [80]:
cursor = con.cursor()
sql = "INSERT INTO solution VALUES (1, 'Miranda Priestly')"
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]

'Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!'